Here we will demonstrate the utility of quantum information through Superdense Coding. Superdense coding is a quantum communication protocol to transmit two classical bits of information from a sender (who will be named Alice) to a receiver (who will be named Bob). The novelty of superdense coding is that Alice will only need to send a single qubit!! The catch is that Alice and Bob will need to pre-share an entangled state. We will create a third party, named Charlie, who is responsible for the creation and sharing of the entangled state.

I am attepting to follow the conventions of Nielsen and Chuang on page 97. You can find free info at https://en.wikipedia.org/wiki/Superdense_coding

I will assume that the reader is familiar with the Pauli matrices, the Hadamard gate, and the CNOT gate.

In [1]:
import numpy as np

# First we should define the gates we will need as numpy arrays. I have chosen the computational basis.

# Single qubit gates
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.eye(2)
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

# Double qubit gates
CNOT = np.array([
    [1,0,0,0],
    [0,0,0,1],
    [0,0,1,0],
    [0,1,0,0]]) # Note that this is the matrix representation when the right most qubit is the control.

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

We will use the common convention that 
$\ket{0} = \begin{bmatrix}
1 \\
0 \end{bmatrix}$
and
$\ket{1} = \begin{bmatrix}
0 \\
1 \end{bmatrix}$

In [2]:
# Define our single qubit states
bra0 = np.array([1,0])
bra1 = np.array([0,1])

Now we turn it over to Charlie to prepare our entangled state.

In [3]:
# Charlie starts with both qubits in the state |00>
psi = np.kron(bra0,bra0)

# Then Charlie applies a Hadamard gate to the rightmost qubit (control)
psi = np.kron(I,H)@psi

# To finish entangling the states, Charlie applies a CNOT gate
psi = CNOT @ psi

# Now Charlie gives the left qubit to Bob and the right qubit to Alice



In [4]:
# Alice sends her message

psi = np.kron(I,Z) @ psi



In [5]:
# Bob decodes the message
psi = CNOT @ psi
psi = np.kron(I,H)@psi

In [6]:
psi

array([0., 1., 0., 0.])